In [4]:
import yaml
import requests
import pandas as pd

In [5]:
API_KEY = yaml.full_load(open('creds.yaml', 'r'))['maps_api']

In [6]:
dd = requests.get('https://gbfs.baywheels.com/gbfs/en/station_information.json')
stations = pd.DataFrame(dd.json()['data']['stations'])

In [5]:
stations.loc[stations['name'].str.contains('Caltrain')]

,station_type,legacy_id,electric_bike_surcharge_waiver,eightd_station_services,short_name,external_id,lon,name,station_id,has_kiosk,capacity,region_id,rental_uris,rental_methods,eightd_has_key_dispenser,lat
128,classic,465,False,[],SF-J29-3,a7da1857-747a-4507-9d8f-2a9f95f0babf,-122.394495,San Francisco Caltrain Station (King St at 4th...,a7da1857-747a-4507-9d8f-2a9f95f0babf,True,18,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.776311
237,classic,130,False,[],SF-P30,e7691d93-eed6-4388-b681-00ae0b455d23,-122.391885,22nd St Caltrain Station,e7691d93-eed6-4388-b681-00ae0b455d23,True,35,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.757704
364,classic,30,False,[],SF-J29,ea11aa7f-c070-46a4-a074-70604340efa2,-122.395282,San Francisco Caltrain (Townsend St at 4th St),ea11aa7f-c070-46a4-a074-70604340efa2,True,19,3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,"[KEY, CREDITCARD]",False,37.776598


In [6]:
lat1, lon1 = stations.loc[stations['name'] == 'Commercial St at Montgomery St'][['lat', 'lon']].iloc[0]

In [7]:
lat2, lon2 = stations.loc[stations['name'] == 'San Francisco Caltrain Station (King St at 4th St)'][['lat', 'lon']].iloc[0]

In [8]:
req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination={},{}&mode=bicycling&origin={},{}&key={}".format(lat1, lon1, lat2, lon2, API_KEY))

In [9]:
res = req.json()

In [10]:
res

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiZsGldZ_j4ARSAUO2zvLtxM',
   'types': ['establishment', 'point_of_interest']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJAyk-84qAhYARoYww6NIvE6A',
   'types': ['establishment', 'point_of_interest']}],
 'routes': [{'bounds': {'northeast': {'lat': 37.7968933, 'lng': -122.3875728},
    'southwest': {'lat': 37.7762883, 'lng': -122.4028589}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '2.4 mi', 'value': 3849},
     'duration': {'text': '15 mins', 'value': 920},
     'end_address': 'Commercial St at Montgomery St, San Francisco, CA 94111, USA',
     'end_location': {'lat': 37.7942628, 'lng': -122.4028589},
     'start_address': 'San Francisco Caltrain Station (King St at 4th St), San Francisco, CA 94107, USA',
     'start_location': {'lat': 37.7762883, 'lng': -122.394524},
     'steps': [{'distance': {'text': '308 ft', 'value': 94},
       'duration': {'text': '1 min', 'value': 17},
 

In [11]:
res.keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [12]:
import datetime as dt

str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))

'0:15:20'

In [13]:
def convert_sec_to_time(num_sec):
    return str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))


def get_travel_time(req):
    return req['routes'][0]['legs'][0]['duration']['value']


def get_stations_coord(name, stations_df):
    if name in stations_df['name'].values:
        return stations_df.loc[stations_df['name'] == name][['lat', 'lon']].iloc[0]
    return None, None

def make_request(station1, station2, stations_df):
    lat1, lon1 = get_stations_coord(station1, stations_df)
    lat2, lon2 = get_stations_coord(station2, stations_df)
    
    req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination={},{}&mode=bicycling&origin={},{}&key={}".format(lat1, lon1, lat2, lon2, API_KEY))
    
    return req.json()

In [14]:
lat, lon = get_stations_coord('Commercial St at Montgomery St', stations)

In [15]:
make_request('Commercial St at Montgomery St', 'San Francisco Caltrain Station (King St at 4th St)', stations)

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiZsGldZ_j4ARSAUO2zvLtxM',
   'types': ['establishment', 'point_of_interest']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJAyk-84qAhYARoYww6NIvE6A',
   'types': ['establishment', 'point_of_interest']}],
 'routes': [{'bounds': {'northeast': {'lat': 37.7968933, 'lng': -122.3875728},
    'southwest': {'lat': 37.7762883, 'lng': -122.4028589}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '2.4 mi', 'value': 3849},
     'duration': {'text': '15 mins', 'value': 920},
     'end_address': 'Commercial St at Montgomery St, San Francisco, CA 94111, USA',
     'end_location': {'lat': 37.7942628, 'lng': -122.4028589},
     'start_address': 'San Francisco Caltrain Station (King St at 4th St), San Francisco, CA 94107, USA',
     'start_location': {'lat': 37.7762883, 'lng': -122.394524},
     'steps': [{'distance': {'text': '308 ft', 'value': 94},
       'duration': {'text': '1 min', 'value': 17},
 

In [16]:
lat1, lon1 = [3.77860194e+01, -1.22403728e+02]

In [2]:
def manhattan_distance(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [3]:
manhattan_distance([3.77860194e+01, -1.22403728e+02], [3.77880194e+01, -1.22423728e+02])

0.021999999999998465

In [11]:
stations.apply(lambda x: manhattan_distance([x.lat, x.lon], [3.77860194e+01, -1.22403728e+02]), axis=1).argmin()

395

In [12]:
stations.iloc[395]

rental_uris                       {'android': 'https://sfo.lft.to/lastmile_qr_sc...
station_id                                     1b13a386-c5f4-42cc-bc3b-ded95982e090
electric_bike_surcharge_waiver                                                False
has_kiosk                                                                      True
region_id                                                                         3
eightd_station_services                                                          []
lon                                                                     -122.405234
capacity                                                                         35
lat                                                                       37.786138
short_name                                                                   SF-G27
eightd_has_key_dispenser                                                      False
name                                   Powell St BART Station (Market St at 

In [10]:
stations

,rental_uris,station_id,electric_bike_surcharge_waiver,has_kiosk,region_id,eightd_station_services,lon,capacity,lat,short_name,eightd_has_key_dispenser,name,external_id,legacy_id,rental_methods,station_type
0,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,0d730ac1-7ce6-45cf-aca6-b412ea46709d,False,True,5,[],-121.902016,27,37.350964,SJ-H10,False,Mission St at 1st St,0d730ac1-7ce6-45cf-aca6-b412ea46709d,288,"[KEY, CREDITCARD]",classic
1,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,ab8cc22e-0f34-4476-bf81-293cbbb2e69c,False,True,5,[],-121.906834,27,37.360854,SJ-F10,False,Kerley Dr at Rosemary St,ab8cc22e-0f34-4476-bf81-293cbbb2e69c,395,"[KEY, CREDITCARD]",classic
2,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,ed707a89-a68d-4921-a4cb-16c268e45a5b,False,True,5,[],-121.883215,23,37.337122,SJ-M11-2,False,San Fernando St at 7th St,ed707a89-a68d-4921-a4cb-16c268e45a5b,280,"[KEY, CREDITCARD]",classic
3,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,dc2f7685-c3e3-4536-b78b-740479cbb207,False,True,5,[],-121.901782,35,37.329732,SJ-M7-1,False,San Jose Diridon Station,dc2f7685-c3e3-4536-b78b-740479cbb207,312,"[KEY, CREDITCARD]",classic
4,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,bae9be55-04d4-4641-9781-3d1c4b6950f1,False,True,5,[],-121.889937,15,37.339301,SJ-L10,False,Saint James Park,bae9be55-04d4-4641-9781-3d1c4b6950f1,306,"[KEY, CREDITCARD]",classic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,f8f12028-2117-4737-9a03-04548ad95e82,False,True,3,[],-122.482786,19,37.781929,SF-G7,False,23rd Ave at Clement St,f8f12028-2117-4737-9a03-04548ad95e82,520,"[KEY, CREDITCARD]",classic
490,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,577,False,True,NaN,[],-122.442729,22,37.799175,SF-C18,False,Divisadero St at Lombard St,82ad4cff-001f-49a8-88fa-a5a63d64b9a2,577,"[KEY, CREDITCARD]",classic
491,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,578,False,True,NaN,[],-122.444305,19,37.730655,SF-X16,False,Detroit St at Hearst Ave,41b2200f-7915-410b-95c6-084bc4d3dee8,578,"[KEY, CREDITCARD]",classic
492,{'android': 'https://sfo.lft.to/lastmile_qr_sc...,579,False,True,5,[],-121.871612,25,37.365088,SJ-H16,False,Berryessa BART Station (Mabury Rd at Lenfest Rd),93d95ded-d49e-43bc-b7fe-6ce556569088,579,"[KEY, CREDITCARD]",classic
